## **Referensi**

 

*   Open AI website (https://openai.com)
*   Lingkungan GYM (http://gym.openai.com/envs/#classic_control)
*   School of AI - Minggu 8 - Reinforcement Learning (https://eliademy.com/app/a/courses/0b3fcfb4bf)
*   Video penjelasan (https://youtu.be/dhFNPQwCEa4)



### install dependencies

Buat Rendering



In [ ]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [ ]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

### ATARI Dependancies

In [ ]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1

## Import Library

In [ ]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML
from statistics import mean,median
from collections import Counter
import keras
from IPython import display as ipythondisplay

In [ ]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

In [ ]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

# Mencoba Game!

In [ ]:
env = wrap_env(gym.make("CartPole-v1"))

In [ ]:
#check out the pacman action space!
print(env.action_space)

In [ ]:
observation = env.reset()

while True:
  
    env.render()
    
    #disini agent
    action = env.action_space.sample() 
         
    observation, reward, done, info = env.step(action) 
    print(observation,reward,done,info)
        
    if done: 
      break;
            
env.close()
show_video()

## Membuat simulasi awal

In [ ]:
import random
goal_step = 500
score_min = 60
iterasi = 10000
training_data = []
scores = []
accepted_scores = []

env = wrap_env(gym.make("CartPole-v1"))

for _ in range(iterasi):
    score = 0
    game_memory = []
    prev_observation = []
    
    env.reset()
    for t in range(goal_step):
      #env.render()
      
      action = random.randrange(0,2)
      observation, reward, done, info = env.step(action) 
      
      if len(prev_observation) > 0:
        game_memory.append([prev_observation, action])
        
      prev_observation = observation
      score += reward
      
      if done:
        break;

    if score >= score_min:
      accepted_scores.append(score)
      for data in game_memory:
        if data[1] == 1:
          output = [0,1]
        
        elif data[1] == 0:
          output = [1,0]
        
        training_data.append([data[0], output])
    
    env.reset()
    scores.append(score)
    
training_data_save = np.array(training_data)
np.save('saved.npy', training_data_save)
  
print('average accepted score',mean(accepted_scores))
print('Median accepted score', median(accepted_scores))
print(Counter(accepted_scores))
  
      
env.close()
#show_video()

print("Panjang data training:" , len(training_data))

## TRAINING JARINGAN SYARAF TIRUAN - OFFLINE

In [ ]:
def build_model(input_size, output_size):
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(128, input_dim=input_size, activation='relu'))
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(256, activation='relu'))
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(512, activation='relu'))
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(256, activation='relu'))
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(128, activation='relu'))
    model.add(keras.layers.Dense(output_size, activation='linear'))
    model.compile(loss='mse', optimizer=keras.optimizers.Adam())
    return model

In [ ]:
def train_model(training_data):
    X = np.array([i[0] for i in training_data]).reshape(-1, len(training_data[0][0]))
    y = np.array([i[1] for i in training_data]).reshape(-1, len(training_data[0][1]))
    model = build_model(input_size=len(X[0]), output_size=len(y[0]))
    
    model.fit(X, y, epochs=14)
    return model

In [ ]:
%%time
tmodel = train_model(training_data)

## GUNAKAN MODEL DI GAME

In [ ]:
iterasi = 10
scores = []
accepted_scores = []
choices = []

env = wrap_env(gym.make("CartPole-v1"))

for _ in range(iterasi):
    score = 0
    game_memory = []
    prev_observation = []
    
    env.reset()
    for t in range(goal_step):
      env.render()
      
      if len(prev_observation) == 0:
        action = random.randrange(0,2)
      else:
        action = np.argmax(tmodel.predict(prev_observation.reshape(-1,len(prev_observation)))[0])
        
      choices.append(action)
      
      new_observation, reward, done, info = env.step(action) 
      prev_observation = new_observation
      score += reward
      
      if done:
        break;
    
    if score > 150:
        #FIX THIS TO CLOSE NOT RESET
        env.close()
        print("This game score:",score)
        show_video()
        break
    
    scores.append(score)
  
print('average accepted score',mean(scores))
print('Median accepted score', median(scores))
print(Counter(choices))
  
      
#env.close()
#show_video()